In [8]:
import os
from nltk.tokenize import sent_tokenize, word_tokenize
import re

indir_pre = os.getcwd() + "/"
outdir_pre = os.getcwd() + "/"
sentence_maxlen = 100


def preprocess(indir):
    # TODO
    # email, Upper-lower case
    # sentence boundary
#     temp = sent_tokenize(indir)
#     output = ""

    buffer,output = "",""
    for root, dirs, filenames in os.walk(indir):
        for f in filenames:
            raw_content = open(os.path.join(root, f),'r').read()
            buffer += raw_content
    temp = sent_tokenize(buffer)
    
    for sent in temp:
        output += "<s> "+sent+" </s> "
    final = remove_punctuation(output)
    return final

#######################################
def remove_punctuation(text):
#     pat = re.compile(r"\p{P}+")
    result = re.findall(r'[\w]+',text)
    delim = " "
    return delim.join(result)

def ngram_generator(n, content):
    def ntoken_count(n, content):
        counter = {}
        tokens = content.split()
        _len = len(tokens)
        for i in xrange(_len - n + 1):
            key = tuple(tokens[i:(i + n)])
            counter[key] = counter.get(key, 0) + 1

        return counter

    counter_n = ntoken_count(n, content)
    prob_dic, hash_dic = {}, {}

    if n == 1:
        _sum = sum(counter_n.values())
        prob_dic = dict((key, num * 1.0 / _sum) for key, num in counter_n.items())
    elif n > 1:
        counter_nminus1 = ntoken_count(n - 1, content)
        for key_n, num_n in counter_n.items():
            key_nminus1 = key_n[:-1]

            hash_dic[key_nminus1] = hash_dic.get(key_nminus1, [])
            hash_dic[key_nminus1].append(key_n)

            num_nminus1 = counter_nminus1[key_nminus1]
            prob_dic[key_n] = 1.0 * num_n / num_nminus1

    return prob_dic, hash_dic


def sentence_generator(prob_dic, hash_dic, pre_sent = ""):
    # TODO:
    pass



def main():
    argv = ["test", "2"] # TODO: input
    if (len(argv) == 0):
        print "Please input a topic"

    topic = argv[0]
    n = int(argv[1]) if len(argv) > 1 and argv[1].isdigit() \
        and int(argv[1]) >= 1 else 1

    indir, outdir = indir_pre + topic, outdir_pre + topic

    if not os.path.isdir(indir):
        print "Sorry, the topic does not exist!"
        return
    if not os.path.isdir(outdir):
        os.makedirs(outdir)

    content = preprocess(indir)
    prob_dic, hash_dic = ngram_generator(n, content)
    print prob_dic
    print hash_dic


if __name__ == "__main__":
    main()


{('to', 'say'): 1.0, ('world!', '</s>'): 1.0, ('hello', 'world!'): 1.0, ('I', 'do'): 1.0, ('beatiful', 'world.'): 1.0, ('world.', '</s>'): 1.0, ('say', 'hello!'): 1.0, ('</s>', '<s>'): 0.6666666666666666, ('hello!', '</s>'): 1.0, ('do', 'like'): 1.0, ('<s>', 'I'): 0.3333333333333333, ('<s>', 'hello'): 0.3333333333333333, ('<s>', 'It'): 0.3333333333333333, ('like', 'to'): 1.0, ('is', 'a'): 1.0, ('It', 'is'): 1.0, ('a', 'beatiful'): 1.0}
{('hello!',): [('hello!', '</s>')], ('<s>',): [('<s>', 'I'), ('<s>', 'hello'), ('<s>', 'It')], ('a',): [('a', 'beatiful')], ('do',): [('do', 'like')], ('world.',): [('world.', '</s>')], ('hello',): [('hello', 'world!')], ('I',): [('I', 'do')], ('is',): [('is', 'a')], ('</s>',): [('</s>', '<s>')], ('like',): [('like', 'to')], ('beatiful',): [('beatiful', 'world.')], ('to',): [('to', 'say')], ('say',): [('say', 'hello!')], ('It',): [('It', 'is')], ('world!',): [('world!', '</s>')]}
